Pipeline for converting a csv input into an output that is suitable for d3 visualization.

In [1]:
import pandas as pd
import numpy as np
from dtaidistance import dtw
from scipy import cluster
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
import os

In [2]:
data = pd.read_csv('./data/sampled_data_5.csv',header=0)

print("There are "+str(len(data['kmer'].unique()))+" unique kmers")
print("data dimension: "+str(data.shape))
data.head()

There are 256 unique kmers
data dimension: (1280, 3)


,read_ID,kmer,values
0,@8a85e853-5be8-4b58-b2d6-c42b920786c1_Basecall...,AACAA,628_671_629_658_673_718_717_698_691_700_710_69...
1,@1d20b26f-6849-41d5-b0b8-0bf9de6ad085_Basecall...,AACAA,664_710_629_654_665_654_649_656_715_707_601_64...
2,@f43fe510-e8e9-4565-a75c-8aa075bbadf8_Basecall...,AACAA,568_568_585_556_583_566_589_568_566_545_541_53...
3,@a360a391-486c-468d-8a60-75d72ce55459_Basecall...,AACAA,635_633_653_623_635_650_635_627_633_607_621_53...
4,@f3365a14-afe9-480b-8fe9-5754ead63217_Basecall...,AACAA,664_629_655_665_652_663_654_654_654_667_650_68...


In [3]:
series_data = []
for i in data.index:
    val = map(int,data.loc[i,'values'].split("_"))
    series_data.append(np.array(list(val), dtype=np.double))

In [4]:
ds = dtw.distance_matrix_fast(series_data,show_progress=True)

In [5]:
ds[np.tril_indices(ds.shape[0],k=-1)] = ds.T[np.tril_indices(ds.shape[0],k=-1)]
np.fill_diagonal(ds,0)
ds = pd.DataFrame(ds)
ds.index = data['kmer']
ds.columns = data['kmer']

In [6]:
for kmer_row in data['kmer'].unique():
    for kmer_column in data['kmer'].unique():
        current_ds = ds.loc[kmer_row, kmer_column]
        current_ds.columns = range(0,5)
        outdir = './d3/data/distance_matrices/'+ kmer_row + '/'
        if not os.path.exists(outdir):
            os.mkdir(outdir)
        filename = kmer_row + '_' + kmer_column + '.csv'
        fullname = os.path.join(outdir, filename)
        
        current_ds.to_csv(fullname, index=False)


In [ ]:
#y = ds[np.triu_indices(ds.shape[0],k=1)]

In [ ]:
#clusters = linkage(y,method='complete')


In [ ]:
"""
row_dendr = dendrogram(clusters)
plt.tight_layout()
plt.ylabel('DTW distance')
plt.xticks(rotation=90)
plt.show()
"""

In [ ]:
"""
# n_clusters is the number of clusters we want to see
for p in range(2,31):
    n_clusters = p
    total_rows = data.shape[0]
    cutree = cluster.hierarchy.cut_tree(Z=clusters,n_clusters=n_clusters)

    cluster_list = []
    percent_list = []

    for i in range(0,n_clusters):
        cluster_list.append(i)
        n_members = len(np.where(cutree==i)[0])
        percentage = n_members / total_rows
        percent_list.append(percentage)
    
    # create the percentage dataframe and write a csv file
    percent_d = {'Cluster':cluster_list,'Percentage':percent_list}
    percent_df = pd.DataFrame(data = percent_d)
    filepath = "./pipeline_output/percentage/percent_df_"+str(p)+".csv"
    percent_df.to_csv(filepath,index=False)
    
    # create the info dataframe and write a csv file
    info_df = data
    info_df['cluster']=0
    for i in range(0,n_clusters):
        info_df.loc[np.where(cutree==i)[0],'cluster']=i
    filepath = "./pipeline_output/info/info_df_"+str(p)+".csv"    
    info_df.to_csv(filepath, index=False)
    
""""